# 🎯 排序模型训练与评估模块 (2_rank.ipynb)

## 📋 模块功能
实现**Learning to Rank**排序模型，将召回的候选商品进行精准排序，提升推荐质量。

## 🚀 核心功能
1. **🔧 特征工程**: 构造用户-商品交互特征
2. **🤖 模型训练**: LightGBM优先，RandomForest备选
3. **📊 模型评估**: Recall@50、NDCG@50等标准指标
4. **🔬 消融实验**: 量化各组件的贡献度
5. **💾 模型保存**: 完整的模型持久化

## 🔬 消融实验设计
- **A1**: 仅协同过滤 vs 多路召回（无排序）
- **A2**: 多路召回无排序 vs 多路召回+排序模型

## 🔧 输出文件
### 评估结果
- `metrics_ablation.csv`: 消融实验指标对比
- `feature_importance.csv`: 特征重要性分析

### 预测结果
- `mm_norank.parquet`: 多路召回无排序预测
- `mm_rank.parquet`: 多路召回+排序预测  
- `cm_norank.parquet`: 协同过滤无排序预测

### 模型文件
- `model_lgb.pkl`: 训练好的排序模型（供线上使用）

## 1️⃣ 环境配置与数据加载

In [11]:
# =============================================================================
# 环境配置
# =============================================================================
import pandas as pd, numpy as np, os, joblib, sys
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

# 添加项目根目录到路径
sys.path.append('..')

# 输出目录
OUTDIR = "../x"
os.makedirs(OUTDIR, exist_ok=True)

# 检查LightGBM
try:
    from lightgbm import LGBMClassifier
    print("✅ LightGBM 可用")
except Exception:
    print("⚠️ LightGBM 不可用，将使用RandomForest")

print(f"📁 输出目录: {OUTDIR}")
print(f"⏰ 开始时间: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}")
print("✅ 环境配置完成")


✅ LightGBM 可用
📁 输出目录: ../x
⏰ 开始时间: 2025-09-26 17:40:39
✅ 环境配置完成


In [12]:
# =============================================================================
# 数据加载（使用1_recall的采样结果）
# =============================================================================
print("📊 开始加载数据...")

# 检查是否有采样后的数据
if os.path.exists(f"{OUTDIR}/train_vis_sampled.parquet"):
    print("✅ 发现采样后的数据，使用采样结果")
    train_vis = pd.read_parquet(f"{OUTDIR}/train_vis_sampled.parquet")
    label_df = pd.read_parquet(f"{OUTDIR}/label_df_sampled.parquet")
    print(f"  train_vis (采样): {len(train_vis):,} 条记录, {train_vis['buyer_admin_id'].nunique():,} 用户")
    print(f"  label_df (采样): {len(label_df):,} 条记录")
else:
    print("⚠️ 未发现采样数据，使用全量数据")
    train_vis = pd.read_parquet(f"{OUTDIR}/train_vis.parquet")
    label_df = pd.read_parquet(f"{OUTDIR}/label_df.parquet")
    print(f"  train_vis (全量): {len(train_vis):,} 条记录, {train_vis['buyer_admin_id'].nunique():,} 用户")
    print(f"  label_df (全量): {len(label_df):,} 条记录")

# 加载召回结果（这些是1_recall生成的）
cands_multi = pd.read_parquet(f"{OUTDIR}/cands_multi.parquet")
cands_covisit = pd.read_parquet(f"{OUTDIR}/cands_covisit_only.parquet")

print(f"  cands_multi: {len(cands_multi):,} 条记录")
print(f"  cands_covisit: {len(cands_covisit):,} 条记录")
print("✅ 数据加载完成")


📊 开始加载数据...
✅ 发现采样后的数据，使用采样结果
  train_vis (采样): 160,699 条记录, 10,000 用户
  label_df (采样): 10,000 条记录
  cands_multi: 1,000,000 条记录
  cands_covisit: 96,959 条记录
✅ 数据加载完成


## 2️⃣ 评估函数定义

In [13]:
# =============================================================================
# 导入留一验证评估函数
# =============================================================================
from leave_one_out_eval import (
    calculate_hr_at_k, calculate_mrr_at_k, calculate_ndcg_at_k,
    evaluate_recommendations, validate_leave_one_out_setup, create_evaluation_report
)

print("✅ 留一验证评估函数已导入")


✅ 留一验证评估函数已导入


In [14]:
# =============================================================================
    # 修复：label_df中的列名是'label_item'，需要重命名为'item_id'
# 特征工程函数
# =============================================================================

def add_label(df):
    """
    添加标签列
    
    Args:
        df: 候选商品DataFrame
    
    Returns:
        添加了label列的DataFrame
    """
    # 合并标签数据
    df_with_label = df.merge(
        label_df[['buyer_admin_id', 'label_item']].rename(columns={'label_item': 'item_id'}).assign(label=1),
        on=['buyer_admin_id', 'item_id'],
        how='left'
    )
    
    # 填充缺失的标签为0
    df_with_label['label'] = df_with_label['label'].fillna(0).astype(int)
    
    return df_with_label

def add_common_feats(df):
    """
    添加通用特征
    
    Args:
        df: 候选商品DataFrame
    
    Returns:
        添加了特征的DataFrame
    """
    # 用户历史购买次数
    user_cnt = train_vis.groupby('buyer_admin_id').size().rename('user_hist_cnt')
    
    # 商品流行度
    item_cnt = train_vis.groupby('item_id').size().rename('item_pop_cnt')
    
    # 合并特征
    df_feat = df.merge(user_cnt, left_on='buyer_admin_id', right_index=True, how='left')
    df_feat = df_feat.merge(item_cnt, left_on='item_id', right_index=True, how='left')
    
    # 填充缺失值
    df_feat['user_hist_cnt'] = df_feat['user_hist_cnt'].fillna(0).astype(int)
    df_feat['item_pop_cnt'] = df_feat['item_pop_cnt'].fillna(0).astype(int)
    
    # 添加来源计数特征
    df_feat['src_count'] = df_feat[['score_rebuy', 'score_covisit', 'is_cate_hot', 'is_store_hot', 'is_global_pop']].notna().sum(axis=1)
    
    return df_feat

print("✅ 特征工程函数已定义: add_label, add_common_feats")


✅ 特征工程函数已定义: add_label, add_common_feats


In [15]:

# === 准备数据 ===
print("📊 准备评估数据...")

# 验证留一验证设置
if not validate_leave_one_out_setup(train_vis, label_df):
    print("❌ 留一验证设置有问题，请检查数据")
    raise ValueError("留一验证设置不正确")

# 准备候选数据
cm = add_common_feats(add_label(cands_covisit.copy()))
mm = add_common_feats(add_label(cands_multi.copy()))

print(f"📊 协同过滤候选: {len(cm):,} 条")
print(f"📊 多路召回候选: {len(mm):,} 条")

# A1: No-Rank baselines（pred = pre_score）
cm_nr = cm.copy(); cm_nr['pred'] = cm_nr['pre_score']
mm_nr = mm.copy(); mm_nr['pred'] = mm_nr['pre_score']

print("🔍 开始A1评估（无排序基线）...")

# 使用正确的留一验证评估
cm_results = evaluate_recommendations(cm_nr, label_df, k_values=[10, 20, 50])
mm_results = evaluate_recommendations(mm_nr, label_df, k_values=[10, 20, 50])

mA1 = {
    'covisit_only_NoRank': {
        'hr50': round(cm_results['hr50'], 6),
        'mrr50': round(cm_results['mrr50'], 6),
        'ndcg50': round(cm_results['ndcg50'], 6),
        'total_score': round(cm_results['total_score'], 6)
    },
    'multi_NoRank': {
        'hr50': round(mm_results['hr50'], 6),
        'mrr50': round(mm_results['mrr50'], 6),
        'ndcg50': round(mm_results['ndcg50'], 6),
        'total_score': round(mm_results['total_score'], 6)
    }
}

print("✅ A1评估完成")
print(f"📊 协同过滤: HR@50={mA1['covisit_only_NoRank']['hr50']:.4f}, 总分={mA1['covisit_only_NoRank']['total_score']:.4f}")
print(f"📊 多路召回: HR@50={mA1['multi_NoRank']['hr50']:.4f}, 总分={mA1['multi_NoRank']['total_score']:.4f}")
    

📊 准备评估数据...
🔍 验证留一验证设置...
⚠️  警告：5402 个标签商品不在训练数据中 (58.3%)
   这在抽样场景下是正常的，因为抽样只选择了部分用户
   共同商品数: 3,861 个
✅ 留一验证设置正确
📊 协同过滤候选: 96,959 条
📊 多路召回候选: 1,000,000 条
🔍 开始A1评估（无排序基线）...
📊 开始综合评估...
  🔍 计算K=10的指标...
    HR@10: 0.1122
    MRR@10: 0.0449
    NDCG@10: 0.0607
  🔍 计算K=20的指标...
    HR@20: 0.1289
    MRR@20: 0.0460
    NDCG@20: 0.0648
  🔍 计算K=50的指标...
    HR@50: 0.1371
    MRR@50: 0.0463
    NDCG@50: 0.0665
✅ 评估完成!
📊 综合评分: 0.1038
👥 评估用户数: 10,000
🎯 候选商品数: 96,959
📊 开始综合评估...
  🔍 计算K=10的指标...
    HR@10: 0.1946
    MRR@10: 0.0869
    NDCG@10: 0.1120
  🔍 计算K=20的指标...
    HR@20: 0.2302
    MRR@20: 0.0895
    NDCG@20: 0.1211
  🔍 计算K=50的指标...
    HR@50: 0.2478
    MRR@50: 0.0901
    NDCG@50: 0.1247
✅ 评估完成!
📊 综合评分: 0.1899
👥 评估用户数: 10,000
🎯 候选商品数: 1,000,000
✅ A1评估完成
📊 协同过滤: HR@50=0.1371, 总分=0.1038
📊 多路召回: HR@50=0.2478, 总分=0.1899


In [16]:
# =============================================================================
# 重新加载模块（解决缓存问题）
# =============================================================================
import importlib
import leave_one_out_eval
importlib.reload(leave_one_out_eval)

# 重新导入更新后的函数
from leave_one_out_eval import (
    calculate_hr_at_k, calculate_mrr_at_k, calculate_ndcg_at_k,
    evaluate_recommendations, validate_leave_one_out_setup, create_evaluation_report
)

print("✅ 模块重新加载完成，使用最新版本的留一验证函数")


✅ 模块重新加载完成，使用最新版本的留一验证函数


In [17]:

# === A2: Multi + Ranker ===
print("🔍 开始A2评估（多路召回+排序模型）...")

feat_cols = ['score_rebuy','score_covisit','is_cate_hot','is_store_hot','is_global_pop',
             'src_count','user_hist_cnt','item_pop_cnt','pre_score']

# LightGBM 优先，无法使用则降级 RF
try:
    from lightgbm import LGBMClassifier
    has_lgbm = True
    print("✅ 使用LightGBM排序模型")
except Exception:
    has_lgbm = False
    from sklearn.ensemble import RandomForestClassifier as RFC
    print("⚠️ 使用RandomForest排序模型")

if has_lgbm:
    model = LGBMClassifier(n_estimators=400, learning_rate=0.05,
                           num_leaves=63, subsample=0.8, colsample_bytree=0.8,
                           random_state=2025, verbose=-1)
else:
    model = RFC(n_estimators=300, random_state=2025, n_jobs=-1)

print("🤖 训练排序模型...")
X = mm[feat_cols].values
y = mm['label'].values
model.fit(X, y)

print("📊 生成排序预测...")
mm_rank = mm.copy()
if hasattr(model, 'predict_proba'):
    mm_rank['pred'] = model.predict_proba(mm[feat_cols])[:,1]
elif hasattr(model, 'decision_function'):
    v = model.decision_function(mm[feat_cols])
    mm_rank['pred'] = (v - v.min())/(v.max()-v.min()+1e-9)
else:
    mm_rank['pred'] = model.predict(mm[feat_cols]).astype(float)

# 使用正确的留一验证评估
print("📊 评估排序模型...")
mm_rank_results = evaluate_recommendations(mm_rank, label_df, k_values=[10, 20, 50])

mA2 = {
    'multi_Ranker': {
        'hr50': round(mm_rank_results['hr50'], 6),
        'mrr50': round(mm_rank_results['mrr50'], 6),
        'ndcg50': round(mm_rank_results['ndcg50'], 6),
        'total_score': round(mm_rank_results['total_score'], 6)
    }
}

print("✅ A2评估完成")
print(f"📊 多路召回+排序: HR@50={mA2['multi_Ranker']['hr50']:.4f}, 总分={mA2['multi_Ranker']['total_score']:.4f}")
    

🔍 开始A2评估（多路召回+排序模型）...
✅ 使用LightGBM排序模型
🤖 训练排序模型...
📊 生成排序预测...
📊 评估排序模型...
📊 开始综合评估...
  🔍 计算K=10的指标...
    HR@10: 0.2644
    MRR@10: 0.2224
    NDCG@10: 0.2326
  🔍 计算K=20的指标...
    HR@20: 0.2724
    MRR@20: 0.2230
    NDCG@20: 0.2347
  🔍 计算K=50的指标...
    HR@50: 0.2788
    MRR@50: 0.2232
    NDCG@50: 0.2360
✅ 评估完成!
📊 综合评分: 0.2585
👥 评估用户数: 10,000
🎯 候选商品数: 1,000,000
✅ A2评估完成
📊 多路召回+排序: HR@50=0.2788, 总分=0.2585


In [18]:
# === 准备数据（修复版本） ===
print("📊 准备评估数据...")

# 重新加载模块（确保使用最新版本）
import importlib
import leave_one_out_eval
importlib.reload(leave_one_out_eval)
from leave_one_out_eval import validate_leave_one_out_setup

# 验证留一验证设置
if not validate_leave_one_out_setup(train_vis, label_df):
    print("❌ 留一验证设置有问题，请检查数据")
    raise ValueError("留一验证设置不正确")

# 准备候选数据
cm = add_common_feats(add_label(cands_covisit.copy()))
mm = add_common_feats(add_label(cands_multi.copy()))

print("✅ 数据准备完成")


📊 准备评估数据...
🔍 验证留一验证设置...
⚠️  警告：5402 个标签商品不在训练数据中 (58.3%)
   这在抽样场景下是正常的，因为抽样只选择了部分用户
   共同商品数: 3,861 个
✅ 留一验证设置正确
✅ 数据准备完成


In [19]:

# === 保存指标、明细、模型、特征重要性 ===
print("💾 保存评估结果...")

# 创建综合指标表
metrics_data = []
for setting, results in mA1.items():
    metrics_data.append({
        'setting': f'A1_{setting}',
        'hr50': results['hr50'],
        'mrr50': results['mrr50'],
        'ndcg50': results['ndcg50'],
        'total_score': results['total_score']
    })

for setting, results in mA2.items():
    metrics_data.append({
        'setting': f'A2_{setting}',
        'hr50': results['hr50'],
        'mrr50': results['mrr50'],
        'ndcg50': results['ndcg50'],
        'total_score': results['total_score']
    })

metrics = pd.DataFrame(metrics_data)
metrics.to_csv(f'{OUTDIR}/metrics_ablation.csv', index=False)
print(f"✅ 消融实验指标已保存: metrics_ablation.csv")

# 保存预测结果
mm_nr.to_parquet(f'{OUTDIR}/mm_norank.parquet', index=False)
mm_rank.to_parquet(f'{OUTDIR}/mm_rank.parquet', index=False)
cm_nr.to_parquet(f'{OUTDIR}/cm_norank.parquet', index=False)
print("✅ 预测结果已保存")

# 保存模型（统一文件名，便于 4_online 自动加载）
model_path = os.path.join(OUTDIR, 'model_lgb.pkl')
joblib.dump(model, model_path)
print(f'✅ 排序模型已保存: {model_path}')

# 特征重要性（若可用）
fi_path = os.path.join(OUTDIR, 'feature_importance.csv')
if hasattr(model, 'feature_importances_'):
    importances = getattr(model, 'feature_importances_')
    fi_df = pd.DataFrame({'feature': feat_cols, 'importance': importances})
    fi_df = fi_df.sort_values('importance', ascending=False)
    fi_df.to_csv(fi_path, index=False)
    print(f'✅ 特征重要性已保存: {fi_path}')
else:
    print('⚠️ 模型无特征重要性信息')

# 生成评估报告
print("\n📊 生成评估报告...")
report = create_evaluation_report(mm_rank_results, "多路召回+排序模型")
print(report)

# 保存评估报告
with open(f'{OUTDIR}/evaluation_report.md', 'w', encoding='utf-8') as f:
    f.write(report)
print(f"✅ 评估报告已保存: evaluation_report.md")

print(f"\n📊 最终消融实验结果:")
print(metrics)
    

💾 保存评估结果...
✅ 消融实验指标已保存: metrics_ablation.csv
✅ 预测结果已保存
✅ 排序模型已保存: ../x/model_lgb.pkl
✅ 特征重要性已保存: ../x/feature_importance.csv

📊 生成评估报告...

# 多路召回+排序模型 评估报告

## 📊 性能指标

| 指标 | K=10 | K=20 | K=50 |
|------|------|------|------|
| HR@K | 0.2644 | 0.2724 | 0.2788 |
| MRR@K | 0.2224 | 0.2230 | 0.2232 |
| NDCG@K | 0.2326 | 0.2347 | 0.2360 |

## 📈 综合评分
- **总评分**: 0.2585

## 📊 数据统计
- **评估用户数**: 10,000
- **候选商品数**: 1,000,000
- **平均每用户候选数**: 100.0

## 🎯 评估方法
- **验证策略**: 留一验证 (Leave-One-Out)
- **时序切分**: 每个用户最后一次购买作为标签
- **评估指标**: HR@K, MRR@K, NDCG@K

✅ 评估报告已保存: evaluation_report.md

📊 最终消融实验结果:
                  setting      hr50     mrr50    ndcg50  total_score
0  A1_covisit_only_NoRank  0.137074  0.046309  0.066506     0.103798
1         A1_multi_NoRank  0.247800  0.090065  0.124671     0.189897
2         A2_multi_Ranker  0.278800  0.223163  0.235954     0.258464
